In [1]:
!pip install SpeechRecognition PySimpleGUI transformers torch pyaudio

In [2]:
import speech_recognition as sr
import PySimpleGUI as sg
import threading
from transformers import BartForConditionalGeneration, BartTokenizer
import os
from difflib import SequenceMatcher  # To compute similarity between texts

# Initialize the BART model and tokenizer once globally
model_name = 'facebook/bart-large-cnn'
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

def recognize_speech(window):
    recognizer = sr.Recognizer()
    microphone = sr.Microphone()
    transcribed_text = ""

    with microphone as source:
        recognizer.adjust_for_ambient_noise(source)
        while listening:
            try:
                audio = recognizer.listen(source, timeout=5)  # Adjust timeout as needed
                text = recognizer.recognize_google(audio)
                window.write_event_value('-TEXT-', text + '\n')
                transcribed_text += text + '\n'  # Store the text
            except sr.UnknownValueError:
                pass
            except sr.RequestError as e:
                print("Error:", e)
            except Exception as e:
                print("Unexpected error:", e)
    return transcribed_text

def bart_summarization(paragraph):
    inputs = tokenizer.encode("summarize: " + paragraph, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(inputs, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

def start_listening(window):
    global listening
    listening = True
    threading.Thread(target=recognize_speech, args=(window,), daemon=True).start()
    window['-START-'].update(disabled=True)
    window['-STOP-'].update(disabled=False)

def stop_listening(window):
    global listening
    listening = False
    window['-START-'].update(disabled=False)
    window['-STOP-'].update(disabled=True)

def save_transcription(text):
    save_path = r'D:\Placement Info\transcription.txt'
    
    # Ensure directory exists
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    
    try:
        with open(save_path, 'a') as file:  # Use 'a' for append mode
            file.write(text)
        print(f"Text successfully saved to {save_path}")
    except Exception as e:
        print(f"Failed to save file: {e}")

# Define the file path as a variable
reference_file_path = r'D:\Placement Info\reference.txt'

def read_reference_text(file_path):
    """Reads the manually written reference text from a file."""
    try:
        if os.path.exists(file_path):
            with open(file_path, 'r') as file:
                return file.read()
        else:
            sg.popup(f"Reference file not found at {file_path}")  # Show popup if the file doesn't exist
            return ""
    except Exception as e:
        sg.popup(f"Failed to read reference file: {e}")
        return ""

def calculate_accuracy(transcribed_text, reference_text):
    """Calculates accuracy by comparing transcribed text with the reference text."""
    matcher = SequenceMatcher(None, transcribed_text, reference_text)
    return matcher.ratio() * 100  # Returns similarity as percentage

# Event loop now includes a button for checking accuracy
listening = False

layout = [
    [sg.Text("Live Speech to Text", font=("Arial", 16))],
    [sg.Output(size=(80, 20), key='-OUTPUT-', text_color='black')],
    [sg.Text("Summary:", font=("Arial", 12))],
    [sg.Text(size=(80, 5), key='-SUMMARY-', text_color='black')],
    [sg.Button('Start', key='-START-', size=(10, 1)), sg.Button('Stop', key='-STOP-', size=(10, 1), disabled=True)],
    [sg.Button('Summarize', key='-SUMMARIZE-', size=(10, 1))],  # Added a Summarize button
    [sg.Button('Save Transcript', key='-SAVE-', size=(15, 1))],
    [sg.Button('Show Accuracy', key='-ACCURACY-', size=(15, 1))]  # Button to show accuracy
]

window = sg.Window('Live Speech to Text', layout)

while True:
    event, values = window.read()
    if event == sg.WIN_CLOSED:
        listening = False
        break
    elif event == '-START-':
        start_listening(window)
    elif event == '-STOP-':
        stop_listening(window)
    elif event == '-TEXT-':
        window['-OUTPUT-'].print(values[event])
    elif event == '-SAVE-':  # Save transcript only when this button is clicked
        transcription_text = window['-OUTPUT-'].get()  # Get the text from the Output element
        if transcription_text.strip():
            save_transcription(transcription_text)  # Now, save the transcription
        else:
            print("No transcription to save.")
    elif event == '-SUMMARIZE-':  # Summarization event
        transcription_text = window['-OUTPUT-'].get()  # Get transcribed text
        if transcription_text.strip():
            summarized_text = bart_summarization(transcription_text)
            window['-SUMMARY-'].update(summarized_text)  # Display summary
        else:
            print("No transcription to summarize.")
    elif event == '-ACCURACY-':  # Show accuracy event
        transcription_text = window['-OUTPUT-'].get()  # Get the transcribed text
        if transcription_text.strip():
            reference_text = read_reference_text(reference_file_path)  # Read reference text
            if reference_text.strip():
                accuracy = calculate_accuracy(transcription_text, reference_text)
                sg.popup(f'Accuracy: {accuracy:.2f}%')  # Display accuracy in a popup
            else:
                sg.popup('Reference text not found or is empty.')
        else:
            sg.popup('No transcription available to check accuracy.')

window.close()
